In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import StratifiedShuffleSplit as SSS
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
housing = fetch_california_housing()
housing_data = housing.data
housing_strat = housing_data.copy()
housing_strat = np.c_[housing_strat,np.ceil(housing_data[:,0] / 1.5)]
unique, counts = np.unique(housing_strat[:,8], return_counts=True)
housing_strat = np.c_[housing_strat, housing.target]
split = SSS(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(housing_strat, housing_strat[:,8]):
    strat_train_set = housing_strat[train_index,:]
    strat_test_set = housing_strat[test_index,:]
housing_train_target, housing_test_target = strat_train_set[:,9], strat_test_set[:,9]
housing_train_data, housing_test_data = strat_train_set[:,:9], strat_test_set[:,:9]
scaler = StandardScaler()
scaler.fit(housing_train_data)
housing_train_data = scaler.transform(housing_train_data)
housing_test_data = scaler.transform(housing_test_data)
m, n = housing_train_data.shape
m_test, n_test = housing_test_data.shape
housing_train_data = np.c_[np.ones((m,1)), housing_train_data]
housing_test_data = np.c_[np.ones((m_test,1)), housing_test_data]

In [6]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = (None, n+1), name = "X")
y = tf.placeholder(tf.float32, shape = (None, 1), name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [3]:
saver = tf.train.import_meta_graph(".\my_model_final.ckpt.meta")

In [4]:
with tf.Session() as sess:
    saver.restore(sess, ".\my_model_final.ckpt")
    print("MSE test = ", sess.run(mse, feed_dict = {X : housing_test_data, y : housing_test_target.reshape(-1, 1)}))

INFO:tensorflow:Restoring parameters from .\my_model_final.ckpt


NameError: name 'mse' is not defined